In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from tqdm import tqdm_notebook as tqdm

from scipy.interpolate import RegularGridInterpolator

from mbptycho.code.simulation import reloadSimulation, Simulation
from skimage.feature import register_translation
from skimage.restoration import unwrap_phase
from scipy import io
import copy
import os
import dill
import ast

cmap = copy.copy(mpl.cm.get_cmap('coolwarm'))
cmap.set_bad('black')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
base_path = os.environ['HOME']

In [4]:
strain_type = 'point_inclusion'
data_path =  f'{base_path}/code/mbptycho/experiments/python/{strain_type}_sparse/apodized_probe_weak_peaks'
sim_data_path = f'{data_path}/sim_{strain_type}.pkl'
sample_data_path = f'{data_path}/sample_{strain_type}.pkl'

probes_3d_path = f'{base_path}/code/mbptycho/experiments/matlab/datasets_0821/probes_4_peaks.mat'

save_plots = False

In [9]:
# The "magnitudes_scaling_per_peak" parameter sets the scattering amplitude per peak.

sm = reloadSimulation(sim_data_path, reload_sim=False,
                      reload_sample_only_filename=sample_data_path,
                      save_sample_only_filename=sample_data_path,
                      new_sim_params={'poisson_noise':True, 
                                      'probes_matlab_file':probes_3d_path, 
                                      'n_scan_positions':9,
                                      'npix_scan_shift':6,
                                      'HKL_list':np.array([[1, 0, 0],
                                                           [1, 1, 0],
                                                           [1, 2, 0],
                                                           [2, 1, 0]]),
                                     'magnitudes_scaling_per_peak': np.array([0.00807052, 0.00807052, 0.00516514, 0.00516514]),
                                     'poisson_noise': True},
                      new_extra_sample_params={'strain_type':strain_type, 
                                               'npix_uncertainty_x':5,
                                               'npix_uncertainty_y':5})
sm.sample.Ux_trunc.shape, sm.sample.params.sample_pix_size

Creating new simulation...
Reloading sample from provided file... /home/skandel/code/mbptycho/experiments/python/point_inclusion_sparse/apodized_probe_weak_peaks/sample_point_inclusion.pkl
Sample reloaded.
Magnitude scaling per peak is supplied. Does not apply random scaling.
Loading probe from matlab data...
Loading successful...
Truncated probe shape (162, 168, 20)
Adding poisson noise...
Truncated probe shape (162, 182, 20)
Adding poisson noise...
Truncated probe shape (162, 200, 20)
Adding poisson noise...
Truncated probe shape (162, 190, 20)
Adding poisson noise...
Saving new simulation at /home/skandel/code/mbptycho/experiments/python/point_inclusion_sparse/apodized_probe_weak_peaks/sim_point_inclusion.pkl...


((200, 200, 20), 0.005844363636363636)

In [11]:
smp = sm.simulations_per_peak[0]

In [12]:
rgi = RegularGridInterpolator((smp.y_nw, smp.x_nw, smp.z_nw),
                               smp.rho,
                                          method='linear',
                                          bounds_error=False,
                                          fill_value=0)

In [13]:
interp1 = rgi(smp.nw_rotated_masked_coords.T)

In [14]:
interp1.sum()

(-548.5699684983749-115.90347232014591j)

In [483]:
np.where(np.sum(sm.sample.obj_mask_full, axis=(1, 2)))

(array([ 58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
         71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
         97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
        110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
        123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
        136, 137, 138, 139, 140, 141, 142]),)

In [484]:
non_zero_y

array([ 58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142])

In [485]:
sm.sample.obj_mask_full.shape[2], non_zero_z.max() + pad_z + 1

(100, 61)

In [498]:
non_zero_z - pad_z, nw_zmin

(array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]),
 40)

In [531]:
pad = 1
pad_z = 2
non_zero_z = np.where(np.sum(sm.sample.obj_mask_full, axis=(0, 1)))[0]
nw_zmin = np.maximum(0, non_zero_z.min() - pad_z)
nw_zmax = np.minimum(sm.sample.obj_mask_full.shape[2], non_zero_z.max() + pad_z)
# Ux_trunc = Ux_full[:, :, nw_zmin: nw_zmax]
# Uy_trunc = Uy_full[:, :, nw_zmin: nw_zmax]
# Uz_trunc = Uz_full[:, :, nw_zmin: nw_zmax]
# obj_mask_trunc = obj_mask_full[:, :, nw_zmin: nw_zmax]
# obj_mask_trunc_no_pad = obj_mask_full_no_pad[:, :, nw_zmin: nw_zmax]

non_zero_y = np.where(np.sum(sm.sample.obj_mask_full, axis=(1, 2)))[0]
#nw_ymin = 0
#nw_ymax = sm.sample.obj_mask_full.shape[0]
nw_ymin = np.maximum(0, non_zero_y.min() - pad)
nw_ymax = np.minimum(sm.sample.obj_mask_full.shape[0],
                     non_zero_y.max() + pad)
print(nw_ymin, nw_ymax)

non_zero_y_delta = np.where(np.sum(sm.sample.obj_mask_w_delta, axis=(1, 2)))[0]
nw_ymin_delta = np.maximum(0, non_zero_y_delta.min() - pad)
nw_ymax_delta = np.minimum(sm.sample.obj_mask_w_delta.shape[0],
                                non_zero_y_delta.max() + pad)
#nw_ymin_delta = 0
#nw_ymax_delta = sm.sample.obj_mask_w_delta.shape[0]

non_zero_x = np.where(np.sum(sm.sample.obj_mask_full, axis=(0, 2)))[0]
nw_xmin = np.maximum(0, non_zero_x.min() - pad)
nw_xmax = np.minimum(sm.sample.obj_mask_full.shape[1],
                          non_zero_x.max() + pad)
#nw_xmin = 0
#nw_xmax = sm.sample.obj_mask_full.shape[1]

non_zero_x_delta = np.where(np.sum(sm.sample.obj_mask_w_delta, axis=(0, 2)))[0]
nw_xmin_delta = np.maximum(0, non_zero_x_delta.min() - pad)
nw_xmax_delta = np.minimum(sm.sample.obj_mask_w_delta.shape[1],
                                non_zero_x_delta.max() + pad)
#nw_xmin_delta = 0
#nw_xmax_delta = sm.sample.obj_mask_w_delta.shape[1]

slice = np.s_[nw_ymin: nw_ymax + 1,
        nw_xmin: nw_xmax + 1,
        nw_zmin: nw_zmax + 1]

slice_delta = np.s_[nw_ymin_delta: nw_ymax_delta + 1,
              nw_xmin_delta: nw_xmax_delta + 1,
              nw_zmin: nw_zmax + 1]

Ux_trunc = sm.sample.Ux_full[slice]
Uy_trunc = sm.sample.Uy_full[slice]
Uz_trunc = sm.sample.Uz_full[slice]
obj_mask_trunc_delta = sm.sample.obj_mask_w_delta[slice_delta]
obj_mask_trunc = sm.sample.obj_mask_full[slice]

x_trunc = sm.sample.x_full[nw_xmin: nw_xmax + 1]
y_trunc = sm.sample.y_full[nw_ymin: nw_ymax + 1]
z_trunc = sm.sample.z_full[nw_zmin: nw_zmax + 1]

x_trunc_delta = sm.sample.x_full[nw_xmin_delta: nw_xmax_delta + 1]
y_trunc_delta = sm.sample.y_full[nw_ymin_delta: nw_ymax_delta + 1]
# z_trunc_delta = z_full[nw_zmin: nw_zmax]

YY_trunc, XX_trunc, ZZ_trunc = np.meshgrid(y_trunc, x_trunc, z_trunc,
                                                                  indexing='ij')

57 143


In [533]:
z_trunc.shape, slice

((21,), (slice(57, 144, None), slice(31, 170, None), slice(40, 61, None)))

In [527]:
x_nw=x_trunc
y_nw=y_trunc
z_nw=z_trunc
                                                            

YY_nw, XX_nw, ZZ_nw = np.meshgrid(y_nw, x_nw, z_nw, indexing='ij')

nw_coords_stacked = np.stack((YY_nw.flatten(), XX_nw.flatten(), ZZ_nw.flatten()), axis=0)

# [
#    self.y_nw, self.x_nw, self.z_nw,
#    self.YY_nw, self.XX_nw, self.ZZ_nw,
#    self.nw_coords_stacked
# ] = self.getNumericalWindowCoordinates(self.rho, self.sample_params.sample_pix_size)

nw_rotated_coords_stacked = (smp.rotate_gamma @ smp.rotate_two_theta @
                             smp.rotate_theta @ smp.nw_coords_stacked)

# Identify the rotated coordinates which lie outside the numerical window, and where the field can
# be set to zero.
# If we have prior knowledge about the sample size, then we can use the sample dimensions instead of
# the numerical window dimensions to set the field to zero. This greatly reduces the computational cost.

nw_rotation_mask = smp.getNonzeroMaskAfterRotation(nw_rotated_coords_stacked,
                                                   sm.sample.params.grain_height,
                                                   sm.sample.params.grain_width,
                                                   sm.sample.params.film_thickness,
                                                   sm.sample.params.sample_pix_size)
nw_rotated_masked_coords = nw_rotated_coords_stacked.T[nw_rotation_mask].T
nw_rotation_mask_indices = np.where(nw_rotation_mask)[0]

In [528]:
smp.rho.shape, z_nw.shape

((200, 200, 20), (19,))

In [534]:
smp.rho_full

AttributeError: 'SimulationPerBraggPeak' object has no attribute 'rho_full'

In [530]:
rgi2 = RegularGridInterpolator((y_nw, x_nw, z_nw),
                               smp.rho[nw_ymin:nw_ymax + 1, nw_xmin:nw_xmax + 1],
                                          method='linear',
                                          bounds_error=False,
                                          fill_value=0)

ValueError: There are 19 points and 20 values in dimension 2

In [525]:
import scipy

In [517]:
smp.rho[nw_ymin:nw_ymax, nw_xmin:nw_xmax].size, YY_nw.size, ZZ_nw.size

(237360, 237360, 237360)

In [518]:
rgi.bounds_error = False
rgi2.bounds_error = False

In [519]:
rgi2(nw_rotated_masked_coords.T[3730:3740])

array([ 0.        +0.j        , -0.0072368 +0.00235028j,
       -0.00768061+0.00247797j, -0.00768547+0.00246309j,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ])

In [520]:
rgi(nw_rotated_masked_coords.T[3730:3740])

array([ 0.        +0.j        , -0.0072368 +0.00235028j,
       -0.00768061+0.00247797j, -0.00768547+0.00246309j,
       -0.00639051+0.00204746j, -0.00503483+0.00161312j,
       -0.00367916+0.00117877j,  0.        +0.j        ,
       -0.00221676+0.00071023j, -0.00153667+0.00049234j])

In [468]:
rgi1.grid[1].max(), rgi1.grid[1].min(), rgi2.grid[1].max(), rgi2.grid[1].min()

(1.0, -1.0, 0.39741672727272725, -0.4032610909090909)

In [459]:
sm.sample.obj_mask_full[:,169,:].sum()

0

In [460]:
sm.sample.x_full[169]

0.4032610909090909

In [448]:
non_zero_x

array([ 32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168])

In [452]:
np.where(np.sum(sm.sample.obj_mask_full, axis=(0, 2)))[0]

array([ 32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168])

In [469]:
nw_xmax

169

In [231]:
rot_mat = smp.rotate_gamma @ smp.rotate_two_theta @ smp.rotate_theta

In [236]:
(np.linalg.pinv(rot_mat) @ (nw_rotated_masked_coords.T[3730:3740]).T).T

array([[-0.24546327,  0.39741673,  0.01753309],
       [-0.24546327,  0.39741673,  0.02337745],
       [-0.24546327,  0.39741673,  0.02922182],
       [-0.24546327,  0.39741673,  0.03506618],
       [-0.24546327,  0.39741673,  0.04091055],
       [-0.24546327,  0.39741673,  0.04675491],
       [-0.24546327,  0.39741673,  0.05259927],
       [-0.24546327,  0.40326109,  0.01753309],
       [-0.24546327,  0.40326109,  0.02337745],
       [-0.24546327,  0.40326109,  0.02922182]])

In [242]:
sm.sample.params.grain_width, sm.sample.params.grain_height, sm.sample.params.film_thickness

(0.8, 0.5, 0.1)

In [237]:
nw_rotated_masked_coords.T[3730:3740]

array([[-0.24546327,  0.39427808, -0.05284186],
       [-0.24546327,  0.39530896, -0.04708913],
       [-0.24546327,  0.39633984, -0.0413364 ],
       [-0.24546327,  0.39737073, -0.03558367],
       [-0.24546327,  0.39840161, -0.02983095],
       [-0.24546327,  0.39943249, -0.02407822],
       [-0.24546327,  0.40046338, -0.01832549],
       [-0.24546327,  0.4000308 , -0.05387274],
       [-0.24546327,  0.40106169, -0.04812001],
       [-0.24546327,  0.40209257, -0.04236728]])

In [252]:
rgi._find_indices(nw_rotated_masked_coords.T[3730:3740].T)

([array([57, 57, 57, 57, 57, 57, 57, 57, 57, 57]),
  array([167, 167, 167, 167, 168, 168, 168, 168, 168, 168]),
  array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])],
 [array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
  array([0.46296131, 0.63935055, 0.8157398 , 0.99212904, 0.16851829,
         0.34490753, 0.52129678, 0.4472818 , 0.62367105, 0.80006029]),
  array([0.95849285, 0.94281335, 0.92713384, 0.91145433, 0.89577483,
         0.88009532, 0.86441581, 0.78210361, 0.7664241 , 0.75074459])],
 array([False, False, False, False, False, False, False, False, False,
        False]))

In [256]:
rgi2._find_indices(nw_rotated_masked_coords.T[3730:3740].T)

([array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([136, 136, 136, 136, 136, 136, 136, 136, 136, 136]),
  array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])],
 [array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
  array([0.46296131, 0.63935055, 0.8157398 , 0.99212904, 1.16851829,
         1.34490753, 1.52129678, 1.4472818 , 1.62367105, 1.80006029]),
  array([0.95849285, 0.94281335, 0.92713384, 0.91145433, 0.89577483,
         0.88009532, 0.86441581, 0.78210361, 0.7664241 , 0.75074459])],
 array([False, False, False, False,  True,  True,  True,  True,  True,
         True]))

In [282]:
test_coord = nw_rotated_masked_coords.T[3735:3736]

In [320]:
rgi.bounds_error=True

In [293]:
rgi(test_coord), rgi2(test_coord)

(array([-0.00503483+0.00161312j]), array([0.+0.j]))

In [321]:
rgi._find_indices(test_coord.T), rgi2._find_indices(test_coord.T)

(([array([57]), array([168]), array([5])],
  [array([1.]), array([0.34490753]), array([0.88009532])],
  array([False])),
 ([array([0]), array([136]), array([5])],
  [array([1.]), array([1.34490753]), array([0.88009532])],
  array([ True])))

In [304]:
smp.y_nw[57], smp.x_nw[168], smp.x_nw[167]

(-0.25130763636363634, 0.39741672727272725, 0.3915723636363636)

In [303]:
y_nw[0], x_nw[136], x_nw[137]

(-0.25130763636363634, 0.3915723636363636, 0.39741672727272725)

In [418]:
rgi2.grid[1]

array([-0.40326109, -0.39741673, -0.39157236, -0.385728  , -0.37988364,
       -0.37403927, -0.36819491, -0.36235055, -0.35650618, -0.35066182,
       -0.34481745, -0.33897309, -0.33312873, -0.32728436, -0.32144   ,
       -0.31559564, -0.30975127, -0.30390691, -0.29806255, -0.29221818,
       -0.28637382, -0.28052945, -0.27468509, -0.26884073, -0.26299636,
       -0.257152  , -0.25130764, -0.24546327, -0.23961891, -0.23377455,
       -0.22793018, -0.22208582, -0.21624145, -0.21039709, -0.20455273,
       -0.19870836, -0.192864  , -0.18701964, -0.18117527, -0.17533091,
       -0.16948655, -0.16364218, -0.15779782, -0.15195345, -0.14610909,
       -0.14026473, -0.13442036, -0.128576  , -0.12273164, -0.11688727,
       -0.11104291, -0.10519855, -0.09935418, -0.09350982, -0.08766545,
       -0.08182109, -0.07597673, -0.07013236, -0.064288  , -0.05844364,
       -0.05259927, -0.04675491, -0.04091055, -0.03506618, -0.02922182,
       -0.02337745, -0.01753309, -0.01168873, -0.00584436,  0.  

In [ ]:
sm.z

In [287]:
grid1 = rgi.grid
grid2 = rgi2.grid

In [ ]:
def _find_indices(self, xi):
        # find relevant edges between which xi are situated
        indices = []
        # compute distance to lower edge in unity units
        norm_distances = []
        # check for out of bounds xi
        out_of_bounds = np.zeros((xi.shape[1]), dtype=bool)
        # iterate through dimensions
        for x, grid in zip(xi, self.grid):
            i = np.searchsorted(grid, x) - 1
            i[i < 0] = 0
            i[i > grid.size - 2] = grid.size - 2
            indices.append(i)
            norm_distances.append((x - grid[i]) /
                                  (grid[i + 1] - grid[i]))
            if not self.bounds_error:
                out_of_bounds += x < grid[0]
                out_of_bounds += x > grid[-1]
        return indices, norm_distances, out_of_bounds

In [315]:
i = np.searchsorted(grid2[1], test_coord[0,1]) - 1
print(i)

137


In [319]:
grid2[1].size

138

In [305]:
list(zip(test_coord.T, grid2))[0]

(array([-0.24546327]),
 array([-0.25130764, -0.24546327, -0.23961891, -0.23377455, -0.22793018,
        -0.22208582, -0.21624145, -0.21039709, -0.20455273, -0.19870836,
        -0.192864  , -0.18701964, -0.18117527, -0.17533091, -0.16948655,
        -0.16364218, -0.15779782, -0.15195345, -0.14610909, -0.14026473,
        -0.13442036, -0.128576  , -0.12273164, -0.11688727, -0.11104291,
        -0.10519855, -0.09935418, -0.09350982, -0.08766545, -0.08182109,
        -0.07597673, -0.07013236, -0.064288  , -0.05844364, -0.05259927,
        -0.04675491, -0.04091055, -0.03506618, -0.02922182, -0.02337745,
        -0.01753309, -0.01168873, -0.00584436,  0.        ,  0.00584436,
         0.01168873,  0.01753309,  0.02337745,  0.02922182,  0.03506618,
         0.04091055,  0.04675491,  0.05259927,  0.05844364,  0.064288  ,
         0.07013236,  0.07597673,  0.08182109,  0.08766545,  0.09350982,
         0.09935418,  0.10519855,  0.11104291,  0.11688727,  0.12273164,
         0.128576  ,  0.1344

In [ ]:
np.searchsorted()

In [255]:
interp2 = rgi2(nw_rotated_masked_coords.T[3730:3740])

In [271]:
np.searchsorted(rgi2.grid

(array([-0.25130764, -0.24546327, -0.23961891, -0.23377455, -0.22793018,
        -0.22208582, -0.21624145, -0.21039709, -0.20455273, -0.19870836,
        -0.192864  , -0.18701964, -0.18117527, -0.17533091, -0.16948655,
        -0.16364218, -0.15779782, -0.15195345, -0.14610909, -0.14026473,
        -0.13442036, -0.128576  , -0.12273164, -0.11688727, -0.11104291,
        -0.10519855, -0.09935418, -0.09350982, -0.08766545, -0.08182109,
        -0.07597673, -0.07013236, -0.064288  , -0.05844364, -0.05259927,
        -0.04675491, -0.04091055, -0.03506618, -0.02922182, -0.02337745,
        -0.01753309, -0.01168873, -0.00584436,  0.        ,  0.00584436,
         0.01168873,  0.01753309,  0.02337745,  0.02922182,  0.03506618,
         0.04091055,  0.04675491,  0.05259927,  0.05844364,  0.064288  ,
         0.07013236,  0.07597673,  0.08182109,  0.08766545,  0.09350982,
         0.09935418,  0.10519855,  0.11104291,  0.11688727,  0.12273164,
         0.128576  ,  0.13442036,  0.14026473,  0.1

In [253]:
interp2.shape

(10,)

In [201]:
np.abs(rgi(nw_rotated_masked_coords.T[3730:3740]) - interp2).sum()

0.02028191115112195

In [197]:
np.sum(nw_rotation_mask)

162864

In [198]:
np.sum(np.abs(smp.rho) > 0)

197965

In [167]:
np.size(nw_rotation_mask_indices)

162864

In [155]:
nw_rotation_mask

array([False, False, False, ..., False, False, False])

## Minimal working example for scipy githup

In [385]:
box_xy = np.linspace(-1,1,30)
box_xy_mask = np.abs(box_xy) < 0.6
box_xy_indices = np.where(box_xy_mask)
zvals = (box_xy * box_xy_mask) * (box_xy[:,None] * box_xy_mask[:,None])

In [398]:
nw1 = box_xy
nw2 = box_xy[4:-4]
zvals2 = zvals[4:-4,4:-4]

In [399]:
rgi1 = RegularGridInterpolator((box_xy, box_xy), 
                               zvals, bounds_error=False, fill_value=0)
rgi2 = RegularGridInterpolator((nw2, nw2), 
                               zvals2, bounds_error=False, fill_value=0)

In [409]:
test_arr = np.array([np.random.randn(5000), np.random.randn(5000)]).T

In [410]:
np.abs(rgi1(test_arr) - rgi2(test_arr)).sum()

0.0

In [402]:
rot_arr = (rot_mat[1:,1:] @ test_arr.T).T

In [404]:
rgi1(rot_arr) - rgi2(rot_arr)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])